# Credit Risk

#### Importing Dependencies

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Importing Core Depdencies
import pandas as pd
from path import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Importing Various Machine Learning Models
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

# Importing Logistical Regression
from sklearn.linear_model import LogisticRegression

# Importing Evaluation Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

#### Extracting Data

In [2]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

# Load the data
file_path = Path('resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


#### Encoding Object data types: 
1. home_ownership
2. verification_status
3. issue_d
4. loan_status (target, custom encoding)
5. pymnt_plan
6. initial_list_status
7. next_pymnt_d
8. application_type
9. hardship_flag
10. debt_settlement_flag

In [3]:
le = LabelEncoder()
df1 = df.copy()
df1['home_ownership'] = le.fit_transform(df1['home_ownership'])

df2 = df1.copy()
df2['verification_status'] = le.fit_transform(df2['verification_status'])

df3 = df2.copy()
df3['issue_d'] = le.fit_transform(df3['issue_d'])

df4 = df3.copy()
df4['pymnt_plan'] = le.fit_transform(df4['pymnt_plan'])

df5 = df4.copy()
df5['initial_list_status'] = le.fit_transform(df5['initial_list_status'])

df6 = df5.copy()
df6['next_pymnt_d'] = le.fit_transform(df6['next_pymnt_d'])

df7 = df6.copy()
df7['application_type'] = le.fit_transform(df7['application_type'])

df8 = df7.copy()
df8['hardship_flag'] = le.fit_transform(df8['hardship_flag'])

df9 = df8.copy()
df9['debt_settlement_flag'] = le.fit_transform(df9['debt_settlement_flag']) 
df9.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


#### Custom encoding for target data

In [4]:
df10 = df9.copy()
df10["loan_status_le"] = le.fit_transform(df10["loan_status"])
df10.head()

loan_status_types = {
    "high_risk": 0,
    "low_risk": 1
}

df10["loan_status_risk_type"] = df10["loan_status"].apply(lambda x: loan_status_types[x])
df10.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,loan_status_le,loan_status_risk_type
0,10500.0,0.1719,375.35,3,66000.0,1,2,low_risk,0,27.24,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0,1,1
1,25000.0,0.2000,929.09,1,105000.0,2,2,low_risk,0,20.23,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0,1,1
2,20000.0,0.2000,529.88,1,56000.0,2,2,low_risk,0,24.26,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0,1,1
3,10000.0,0.1640,353.55,3,92000.0,2,2,low_risk,0,31.44,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0,1,1
4,22000.0,0.1474,520.39,1,52000.0,0,2,low_risk,0,18.76,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0,1,1


#### Split the Data into Training and Testing

In [5]:
# Create features
X = df10.copy()
X = X.drop("loan_status", axis=1)

# Create target
y = df10["loan_status"].values

X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,loan_status_le,loan_status_risk_type
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,0.994958,0.994958
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.070831,0.070831
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,1.000000,1.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,1.000000,1.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,1.000000,1.000000


#### Normal train-test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

## Oversampling with RandomOverSampler and SMOTE algorithms

In this section, I will compare two oversampling algorithms to determine which algorithm results in the best performance. I will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

#### implement random oversampling

In [7]:
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

#### Logistic regression using random oversampled data

In [8]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

#### Display the confusion matrix

In [9]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   71,    30],
       [ 6451, 10653]])

#### Calculate the balanced accuracy score

In [10]:
balanced_accuracy_score(y_test, y_pred)

0.6629035301799591

#### Print the imbalanced classification report

In [11]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.62      0.02      0.66      0.44       101
   low_risk       1.00      0.62      0.70      0.77      0.66      0.43     17104

avg / total       0.99      0.62      0.70      0.76      0.66      0.43     17205



#### implement SMOTE algorithm

In [13]:
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

#### Logistic regression using SMOTE Algorithm Data

In [14]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

#### Display the confusion matrix

In [15]:
confusion_matrix(y_test, y_pred)

array([[   71,    30],
       [ 6451, 10653]])

#### Calculate the balanced accuracy score

In [16]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6366387574211116

#### Print the imbalanced classification report

In [17]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.67      0.02      0.64      0.40       101
   low_risk       1.00      0.67      0.60      0.80      0.64      0.41     17104

avg / total       0.99      0.67      0.60      0.80      0.64      0.41     17205



## Undersampling with Cluster Centroids algorithms

In this section, I will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. I will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [18]:
# Normal train-test split

#### Undersample the data using `RandomUnderSampler`

In [19]:
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

#### Fit a Logistic regression model using random undersampled data

In [20]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

#### Display the confusion matrix

In [21]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  65,   36],
       [7302, 9802]])

#### Calculate the Balanced Accuracy Score

In [22]:
balanced_accuracy_score(y_test, y_pred)

0.6083233381804036

#### Print the imbalanced classification report

In [23]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.57      0.02      0.61      0.37       101
   low_risk       1.00      0.57      0.64      0.73      0.61      0.37     17104

avg / total       0.99      0.57      0.64      0.72      0.61      0.37     17205



#### Fit the data using `ClusterCentroids` and check the count of each class

In [24]:
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

#### Logistic regression using cluster centroid undersampled data

In [25]:
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

#### Display the confusion matrix

In [26]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  66,   35],
       [9989, 7115]])

#### Calculate the balanced accuracy score

In [27]:
balanced_accuracy_score(y_test, y_pred)

0.5347249557743426

#### Print the imbalanced classification report

In [28]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.65      0.42      0.01      0.52      0.28       101
   low_risk       1.00      0.42      0.65      0.59      0.52      0.27     17104

avg / total       0.99      0.42      0.65      0.58      0.52      0.27     17205



## Combination approach with SMOTEEN algorithms

In this section, I will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. I will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

#### Normal train-test split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#### Use the SMOTEENN technique to perform combination sampling on the data, Count the resampled classes

In [30]:
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 68460, 'low_risk': 62011})

#### Fit a Logistic regression model using random undersampled data

In [31]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

#### Display the confusion matrix

In [32]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  78,   23],
       [8073, 9031]])

#### Calculate the Balanced Accuracy Score

In [33]:
balanced_accuracy_score(y_test, y_pred)

0.6501411863590475

#### Print the imbalanced classification report

In [34]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.77      0.53      0.02      0.64      0.42       101
   low_risk       1.00      0.53      0.77      0.69      0.64      0.40     17104

avg / total       0.99      0.53      0.77      0.69      0.64      0.40     17205

